# Segmenting and Clustering Neighborhoods in Toronto

Importing libraries

In [1]:
import pandas as pd # library for data analysis
import requests
import folium
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from bs4 import BeautifulSoup #for scraping


print('Libraries imported.')

Libraries imported.


<a id='menu1'></a>

## Part 1

We are going to scraping the neighborhood data from Wikipedia. 

In [2]:
wiki = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

r = requests.get(wiki)

In [3]:
soup = BeautifulSoup(r.content)
trs = soup.find('table','wikitable sortable').find_all('tr')
columns = ['postcode', 'borough', 'neighborhood']
df_postcode = pd.DataFrame(columns=columns)
for tr in trs:
    tds = tr.find_all('td')
    if len(tds) == 0: continue
    postcode = tds[0].get_text().strip()
    borough = tds[1].get_text().strip()
    neighborhood = tds[2].get_text().strip()
    if borough != 'Not assigned':
#         print("{} - {} - {}".format(postcode, borough, neighborhood))
        df_postcode = df_postcode.append({'postcode': postcode, 'borough': borough, 'neighborhood':neighborhood}, ignore_index=True)

print(df_postcode.shape[0])
df_postcode.head()

211


,postcode,borough,neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Find ___"Not assigned"___ neighborhood and replace it with borough value

In [4]:
df_postcode[df_postcode.neighborhood=='Not assigned']

,postcode,borough,neighborhood
6,M7A,Queen's Park,Not assigned


In [5]:
df_postcode['neighborhood'] = df_postcode.apply(lambda x: x.borough if x.neighborhood == 'Not assigned' else x.neighborhood, axis=1)

In [6]:
df_postcode[df_postcode.neighborhood=='Not assigned']

,postcode,borough,neighborhood


Now there is no neighborhood with ___"Not Assigned"___ value

Next step is __grouping neighborhood__ with the same postcode into one row

In [7]:
df_postcode_grouped = df_postcode.groupby(['postcode','borough'], as_index=False).count()
df_postcode_grouped
df_postcode_clean = pd.DataFrame(columns=columns)
for index, row in df_postcode_grouped.iterrows():
    neighborhood = df_postcode[df_postcode.postcode==row[0]].neighborhood.str.cat(sep=',')
    df_postcode_clean = df_postcode_clean.append({'postcode':row[0], 'borough':row[1], 'neighborhood':neighborhood}, ignore_index=True)
    
df_postcode_clean

,postcode,borough,neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [ ]:
df_postcode_clean.shape

(103, 3)

## Part 2

Find __coordinate__ of each postcode by reading from https://cocl.us/Geospatial_data

In [ ]:
# read from url
url = 'https://cocl.us/Geospatial_data'
df_coordinate = pd.read_csv(url)

Postcode' coordinates dataframe:

In [ ]:
df_coordinate.head()

__Join__ with df_postcode_clean dataframe

In [ ]:
df_postcode_coordinate = df_postcode_clean.join(df_coordinate.set_index('Postal Code'), on='postcode')
df_postcode_coordinate.head()

In [ ]:
df_postcode_coordinate.shape

__All__ postcode now has coordinates

## Part 3

In [ ]:
map_city = folium.Map(location=[43.6532, -79.3832], zoom_start=10)
# gpd_wards.plot(figsize=(20,20))
# plt.show()

for lat, long, postcode, borough, neighborhood in zip(df_postcode_coordinate.Latitude, df_postcode_coordinate.Longitude, df_postcode_coordinate.postcode, df_postcode_coordinate.borough, df_postcode_coordinate.neighborhood):
        folium.CircleMarker(
            [lat, long],
            radius=5,
            color='yellow',
            fill=True,
            popup="<b>{}</b>\n{}:\n{}".format(postcode, borough, neighborhood),
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(map_city)

map_city

In [ ]:
CLIENT_ID = 'WXNUH2PMIPGTENRW1DBJDNR0YB4PZSG3VKW3EFFMB435QANG' # your Foursquare ID
CLIENT_SECRET = 'IMYN2GET53TGRICI2MGHNN0JJIREVQLOAW1BOI2V0PD2TC3N' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

#### Let's explore the first neighborhood in our dataframe.

In [ ]:
neighborhood_latitude, neighborhood_longitude = df_postcode_coordinate.iloc[0].Latitude, df_postcode_coordinate.iloc[0].Longitude
url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v=20180605&ll={},{}&radius=500&limit=100".format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude)
print(url)

In [ ]:
results = requests.get(url).json()
results

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

## 2. Explore Neighborhoods in City of Toronto

Again borrowing funtion from the course

In [ ]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])


    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
district_venues = getNearbyVenues(names=df_postcode_coordinate['postcode'], latitudes=df_postcode_coordinate['Latitude'],longitudes=df_postcode_coordinate['Longitude'],radius=500)

In [ ]:
district_venues.shape[0]

In [ ]:
district_venues.head()

Let's check how many venues were returned for each neighborhood

In [ ]:
district_venues.groupby('Neighborhood').count()

#### Let's find out how many unique categories can be curated from all the returned venues

In [ ]:
print('There are {} uniques categories.'.format(len(district_venues['Venue Category'].unique())))

<a id='menu3'></a>

## 3. Analyze Each Neighborhood

In [ ]:
# one hot encoding
district_onehot = pd.get_dummies(district_venues[['Venue Category']], prefix="", prefix_sep="")

#Add Neighborhood name
district_onehot = pd.concat([district_venues[['Neighborhood']], district_onehot], axis=1)

district_onehot.head(10)

In [ ]:
district_onehot.info(verbose=True)

And let's examine the new dataframe size.

In [ ]:
district_onehot.shape

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
district_onehot.groupby('Neighborhood').mean().reset_index()

In [ ]:
district_grouped = district_onehot.groupby('Neighborhood').mean().reset_index()
district_grouped.head()

#### Let's confirm the new size

In [ ]:
district_grouped.shape

#### Let's print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in district_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = district_grouped[district_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True)[:5])
    print('\n')

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = district_grouped['Neighborhood']

for ind in np.arange(district_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(district_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

<a id='menu4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [ ]:
# set number of clusters
kclusters = 5

district_grouped_clustering = district_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(district_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

district_merged = df_neighbourhood

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
district_merged = district_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='neighborhood')

district_merged.head() # check the last columns!

Finally, let's visualize the resulting clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(district_merged['latitude'], district_merged['longitude'], district_merged['neighborhood'], district_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='menu5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [ ]:
district_merged.loc[district_merged['Cluster Labels'] == 0, district_merged.columns[[0] + list(range(4, district_merged.shape[1]))]]

#### Cluster 2

In [ ]:
district_merged.loc[district_merged['Cluster Labels'] == 1, district_merged.columns[[0] + list(range(5, district_merged.shape[1]))]]

#### Cluster 3

In [ ]:
district_merged.loc[district_merged['Cluster Labels'] == 2, district_merged.columns[[0] + list(range(5, district_merged.shape[1]))]]

#### Cluster 4

In [ ]:
district_merged.loc[district_merged['Cluster Labels'] == 3, district_merged.columns[[0] + list(range(5, district_merged.shape[1]))]]

#### Cluster 5

In [ ]:
district_merged.loc[district_merged['Cluster Labels'] == 4, district_merged.columns[[0] + list(range(5, district_merged.shape[1]))]]